In [18]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, f1_score, classification_report

import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('../cleaned_data.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1851422 entries, 0 to 1851421
Data columns (total 36 columns):
 #   Column             Dtype  
---  ------             -----  
 0   FlightDate         object 
 1   Quarter            int64  
 2   Year               int64  
 3   Month              int64  
 4   DayofMonth         int64  
 5   DepTime            float64
 6   DepDel15           float64
 7   CRSDepTime         int64  
 8   DepDelayMinutes    float64
 9   OriginAirportID    int64  
 10  DestAirportID      int64  
 11  ArrTime            float64
 12  CRSArrTime         int64  
 13  ArrDelayMinutes    float64
 14  Origin             object 
 15  Dest               object 
 16  RoundedFlightDate  object 
 17  DepatHr            int64  
 18  ArrDel15           float64
 19  Delayed            int64  
 20  date               object 
 21  airport            object 
 22  windspeedKmph      int64  
 23  winddirDegree      int64  
 24  weatherCode        int64  
 25  precipMM          

In [4]:
data = pd.concat([data, pd.get_dummies(data['Origin'],prefix='Origin')], axis=1)
data = pd.concat([data, pd.get_dummies(data['Dest'],prefix='Destination')], axis=1)

In [5]:
Df = data[[ 'Quarter', 'Month', 'DayofMonth',
       'CRSDepTime', 'CRSArrTime', 
       'windspeedKmph', 'winddirDegree', 'weatherCode', 'precipMM',
       'visibility', 'pressure', 'cloudcover', 'DewPointF', 'WindGustKmph',
       'tempF', 'WindChillF', 'humidity',  'Origin_ATL',
       'Origin_CLT', 'Origin_DEN', 'Origin_DFW', 'Origin_EWR', 'Origin_IAH',
       'Origin_JFK', 'Origin_LAS', 'Origin_LAX', 'Origin_MCO', 'Origin_MIA',
       'Origin_ORD', 'Origin_PHX', 'Origin_SEA', 'Origin_SFO',
       'Destination_ATL', 'Destination_CLT', 'Destination_DEN',
       'Destination_DFW', 'Destination_EWR', 'Destination_IAH',
       'Destination_JFK', 'Destination_LAS', 'Destination_LAX',
       'Destination_MCO', 'Destination_MIA', 'Destination_ORD',
       'Destination_PHX', 'Destination_SEA', 'Destination_SFO',  'DepDel15']]


In [6]:
Df.head()

,Quarter,Month,DayofMonth,CRSDepTime,CRSArrTime,windspeedKmph,winddirDegree,weatherCode,precipMM,visibility,...,Destination_JFK,Destination_LAS,Destination_LAX,Destination_MCO,Destination_MIA,Destination_ORD,Destination_PHX,Destination_SEA,Destination_SFO,DepDel15
0,1,1,1,2347,714,15,123,113,0.0,10,...,0,0,0,0,1,0,0,0,0,1.0
1,1,1,1,20,705,15,123,113,0.0,10,...,0,0,0,0,0,0,0,0,0,0.0
2,1,1,1,2359,603,15,38,113,0.0,10,...,0,0,0,0,0,1,0,0,0,0.0
3,1,1,1,30,823,15,38,113,0.0,10,...,0,0,0,0,1,0,0,0,0,0.0
4,1,1,1,10,501,15,38,113,0.0,10,...,0,0,0,0,0,0,0,0,0,0.0


In [14]:
X_train, X_test, y_train, y_test = train_test_split(Df.iloc[:, :-1], Df.iloc[:, -1], train_size=0.75, random_state=0)

In [15]:
X_train.shape, y_train.shape

((1388566, 47), (1388566,))

In [16]:
y_train.value_counts()

0.0    1109362
1.0     279204
Name: DepDel15, dtype: int64

In [17]:
ratio = 1109362/279204
ratio

3.97330267474678

4:1 ratio

In [39]:
weights = {0:1.0, 1:4.0}
cls = DecisionTreeClassifier(random_state=1, class_weight=weights)

In [40]:
cls.fit(X_train.values, y_train.values)

DecisionTreeClassifier(class_weight={0: 1.0, 1: 4.0}, random_state=1)

In [41]:
prediction = cls.predict(X_test.values)
prediction[:100]

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0.])

In [42]:
y_test.values[:100]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
       1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1.,
       0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0.])

In [43]:
print(classification_report(prediction, y_test.values))

              precision    recall  f1-score   support

         0.0       0.86      0.85      0.86    373766
         1.0       0.41      0.43      0.42     89090

    accuracy                           0.77    462856
   macro avg       0.64      0.64      0.64    462856
weighted avg       0.77      0.77      0.77    462856



In [44]:
accuracy_score(prediction, y_test.values)

0.7709028293897022

In [45]:
f1_score(prediction, y_test.values)

0.41724325542286533